Data Analysis & Preprocessing:
- Exploratory Data Analysis (EDA)
- Check class distribution (balanced/imbalanced)
- Handle missing values
- Encode categorical variables
- Feature scaling if needed
- Feature selection/importance analysis


Modeling Strategy:
- Start with simple models as baseline (Logistic Regression)
- Move to more complex models:Random Forest, XGBoost/LightGBM, Support Vector Machines
- Use cross-validation for robust evaluation
- If data is imbalanced, consider: SMOTE/ADASYN for oversampling, Class weights, Ensemble methods

Evaluation Metrics to focus on:
- Accuracy (if balanced classes)
- Precision, Recall, F1-score
- ROC-AUC
- Confusion Matrix

Interpretability:
- Feature importance
- SHAP values
- Partial dependence plots

reference : https://medium.com/data-and-beyond/mastering-exploratory-data-analysis-eda-everything-you-need-to-know-7e3b48d63a95

#### Importing Libraries

In [5]:
# import libraries

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

#### Loading the training dataset

In [11]:
# loading the training dataset

df = pd.read_csv("data/train.csv", index_col="id")

print("data snapshot")
df.head()


data snapshot


,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
id,,,,,,,,,,,,,,,,,,,
0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


PS: set index_col to id if there's already one

#### Concise summary of the training dataframe

In [10]:
# column details

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 20 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  object 
 2   Gender                                 140700 non-null  object 
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  object 
 5   Working Professional or Student        140700 non-null  object 
 6   Profession                             104070 non-null  object 
 7   Academic Pressure                      27897 non-null   float64
 8   Work Pressure                          112782 non-null  float64
 9   CGPA                                   27898 non-null   float64
 10  Study Satisfaction                     27897 non-null   

#### To solve the problem statement in hand, let's highlight few line items 
- "Depression" is the target variable
- probable factors infuencing the target
    1. id - no
    2. Name - no
    3. Gender - yes | indirectly | could be biased towards one or the other category
    4. Age - yes | indirectly | could be more prevalent at certain groups when considering multiple factors like job situation / academic situation etc
    5. City - yes | indirectly | observe the trends for different cities
    6. Working Professional or Student - yes | indirectly | could depend on other factors
    7. Profession - yes | indirectly | could have some relation to the profession
    8. Academic Pressure - yes | directly | could be for students
    9. Work Pressure - yes | directly | could be for employees
    10. CGPA - yes | directly | could be for students
    11. Study Satisfaction - yes | directly | for students
    12. Job Satisfaction - yes | directly | for employees
    13. Sleep Duration - yes | directly | could be directly related
    14. Dietary Habits - yes | indirectly | could depend on other factors
    15. Degree - no
    16. Have you ever had suicidal thoughts ? - yes | directly | most obvious
    17. Work/Study Hours - yes | indirectly | could lead to tiredness and influence other factors
    18. Financial Stress - yes | directly | stress of any kind will influence
    19. Family History of Mental Illness - yes | 
    20. directly | for some genetic relation

#### EDA

Null check

In [13]:
df.isnull().sum()

Name                                          0
Gender                                        0
Age                                           0
City                                          0
Working Professional or Student               0
Profession                                36630
Academic Pressure                        112803
Work Pressure                             27918
CGPA                                     112802
Study Satisfaction                       112803
Job Satisfaction                          27910
Sleep Duration                                0
Dietary Habits                                4
Degree                                        2
Have you ever had suicidal thoughts ?         0
Work/Study Hours                              0
Financial Stress                              4
Family History of Mental Illness              0
Depression                                    0
dtype: int64

- 36630 records don't have profession. Need to inspect what's in-place of the correct value.
- For both academic pressure and work pressure there's no problem as collectively they should be = 140700 and that matches
- CGPA, study satisfaction and job satisfaction are fine
- Dietary habits and Financial stress are missing for 4 individuals. Need to do something

In [3]:
df.describe()

,id,Age,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Work/Study Hours,Financial Stress,Depression
count,140700.000000,140700.000000,27897.000000,112782.000000,27898.000000,27897.000000,112790.000000,140700.000000,140696.000000,140700.000000
mean,70349.500000,40.388621,3.142273,2.998998,7.658636,2.944940,2.974404,6.252679,2.988983,0.181713
std,40616.735775,12.384099,1.380457,1.405771,1.464466,1.360197,1.416078,3.853615,1.413633,0.385609
min,0.000000,18.000000,1.000000,1.000000,5.030000,1.000000,1.000000,0.000000,1.000000,0.000000
25%,35174.750000,29.000000,2.000000,2.000000,6.290000,2.000000,2.000000,3.000000,2.000000,0.000000
50%,70349.500000,42.000000,3.000000,3.000000,7.770000,3.000000,3.000000,6.000000,3.000000,0.000000
75%,105524.250000,51.000000,4.000000,4.000000,8.920000,4.000000,4.000000,10.000000,4.000000,0.000000
max,140699.000000,60.000000,5.000000,5.000000,10.000000,5.000000,5.000000,12.000000,5.000000,1.000000


In [4]:
# column summary

def column_summary(df):
    data = []

    for column in df.columns:
        data_type = df[column].dtype
        null_count = df[column].isnull().sum()
        non_null_count = df[column].notnull().sum()
        distinct_values = df[column].nunique()

        if distinct_values <= 10:
            distinct_value_count = df[column].value_counts().to_dict()
        else:
            top_10_distinct_values = df[column].value_counts().head(10).to_dict()
            distinct_value_count = {k:v for k, v in sorted(top_10_distinct_values.items(), key=lambda item: item[1], reverse=True)}

        data.append({
            "name": column,
            "column_dtype" : data_type,
            "#_null": null_count,
            "#_non_null": non_null_count,
            "unique_values": distinct_values,
            "unique_value_counts": distinct_value_count,
        })

    data_df = pd.DataFrame(data)
    return data_df

data_summary = column_summary(df)
display(data_summary)

,name,column_dtype,#_null,#_non_null,unique_values,unique_value_counts
0,id,int64,0,140700,140700,"{0: 1, 93803: 1, 93797: 1, 93798: 1, 93799: 1, 93800: 1, 93801: 1, 93802: 1, 93804: 1, 93778: 1}"
1,Name,object,0,140700,422,"{'Rohan': 3178, 'Aarav': 2336, 'Rupak': 2176, 'Aaradhya': 2045, 'Anvi': 2035, 'Raghavendra': 1877, 'Vani': 1657, 'Tushar': 1596, 'Ritvik': 1589, 'Shiv': 1568}"
2,Gender,object,0,140700,2,"{'Male': 77464, 'Female': 63236}"
3,Age,float64,0,140700,43,"{56.0: 5246, 49.0: 5099, 38.0: 4564, 53.0: 4526, 57.0: 4395, 47.0: 4199, 46.0: 4080, 54.0: 3928, 51.0: 3927, 18.0: 3921}"
4,City,object,0,140700,98,"{'Kalyan': 6591, 'Patna': 5924, 'Vasai-Virar': 5765, 'Kolkata': 5689, 'Ahmedabad': 5613, 'Meerut': 5528, 'Ludhiana': 5226, 'Pune': 5210, 'Rajkot': 5207, 'Visakhapatnam': 5176}"
5,Working Professional or Student,object,0,140700,2,"{'Working Professional': 112799, 'Student': 27901}"
6,Profession,object,36630,104070,64,"{'Teacher': 24906, 'Content Writer': 7814, 'Architect': 4370, 'Consultant': 4229, 'HR Manager': 4022, 'Pharmacist': 3893, 'Doctor': 3255, 'Business Analyst': 3161, 'Entrepreneur': 2968, 'Chemist': 2967}"
7,Academic Pressure,float64,112803,27897,5,"{3.0: 7463, 5.0: 6296, 4.0: 5158, 1.0: 4801, 2.0: 4179}"
8,Work Pressure,float64,27918,112782,5,"{2.0: 24373, 4.0: 22512, 5.0: 22436, 3.0: 21899, 1.0: 21562}"
9,CGPA,float64,112802,27898,331,"{8.04: 822, 9.96: 425, 5.74: 410, 8.95: 371, 9.21: 343, 7.25: 339, 7.09: 320, 7.88: 318, 9.44: 317, 8.91: 276}"
